# 1. Import Packages & Set ChromeDriver

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 5.9MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launch

In [3]:
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import os
import requests
from bs4 import BeautifulSoup

import pandas as pd
pd.set_option('display.max_rows', 900)
pd.set_option('display.max_columns', 900)

In [4]:
cd /content/drive/MyDrive/TobigSlang

/content/drive/.shortcut-targets-by-id/1Tys5-QaX-3ICj4J5S283IJYXj7FSQFAe/TobigSlang


In [5]:
cd '2. Modeling '/'정진모'/'미경'

/content/drive/.shortcut-targets-by-id/1Tys5-QaX-3ICj4J5S283IJYXj7FSQFAe/TobigSlang/2. Modeling /정진모/미경


In [6]:
cd models/'#preprocessing'

/content/drive/.shortcut-targets-by-id/1Tys5-QaX-3ICj4J5S283IJYXj7FSQFAe/TobigSlang/2. Modeling /정진모/미경/models/#preprocessing


In [7]:
ls

'1227data(1)'/   crawling_1227_colab.ipynb   preprocess.ipynb
 backup/         data/                       preprocess_module.ipynb
 chromedriver    kin_1227.csv


In [8]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


# 2. Crawler

## 2-1. Crawling doctor's list
https://kin.naver.com/people/expert/index.nhn?type=DOCTOR&edirId=11&orgId=0&sort=answerCount  
[진료과목] - 정신건강의학  
[정렬기준] - 답변개수순  
답변개수가 1개 이상인 의사들의 프로필 링크를 수집하여 doctors에 저장

In [ ]:
# 답변개수순으로 정렬했을 때 9페이지의 4번째 항목까지가 해당됨. 따라서 9페이지까지 수집하고 이후에 답변이 없는 행은 제외시키는 방식으로 진행.
# 의사 이름, 소속, 프로필url을 순서대로 수집함.
doc_names = []
doc_instits = []
doc_anscount= []
doc_profiles = []
 

for i in range(1, 10):
    print("---------------------------",i,"page---------------------------")
    url = "https://kin.naver.com/people/expert/index.nhn?orgId=0&sort=answerCount&edirId=11&type=DOCTOR&page=" + str(i)
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        for i in range(1, 11):
            print("******",i,"******")
            name = soup.select_one("#content > div.pro_listbox > ul > li:nth-of-type(" + str(i) + ") > dl > dd > h5 > a")
            institute = soup.select_one("#content > div.pro_listbox > ul > li:nth-of-type(" + str(i) + ") > dl > dd > h6")
            anstable = soup.select_one("#content > div.pro_listbox > ul > li:nth-of-type(" + str(i) + ") > dl > dd > table")
            anscount = anstable.select_one("em")
            profile = name.get('href')

            try:
                name = name.get_text().strip()
                print("[Name]",name)
                doc_names.append(name)
                
                print("[Profile]",profile)
                doc_profiles.append(profile)
                
                anscount = anscount.get_text().strip()
                print("[anscount]",anscount)
                doc_anscount.append(anscount)
                
                institute = institute.get_text().strip()
                print("[Institute]",institute,"\n")
                doc_instits.append(institute)
                
            except:
                print("[Error!] Institute data does not exist.\n")
                institute = ""
                doc_instits.append(institute)
                
    else:
        print("Connect Failed")
        print(response.status_code)

--------------------------- 1 page---------------------------
****** 1 ******
[Name] 신홍범
[Profile] /userinfo/expert/index.nhn?u=VYRdkbmstCwDtA0tX7VxvE3j7JfcoiA%2Fvj0ZoFMcqks%3D
[anscount] 23,624
[Institute] 코슬립수면의원 대표원장 

****** 2 ******
[Name] 권순모
[Profile] /userinfo/expert/index.nhn?u=QbPYSQHbEVHsy13frYO7Wkq9LJla1qotlZkVSwYDsqk%3D
[anscount] 3,626
[Error!] Institute data does not exist.

****** 3 ******
[Name] 배성범
[Profile] /userinfo/expert/index.nhn?u=HaNfIgmblpoaWZLW%2FUqzLu7EcB%2BNPc9ybXr1CIjefDI%3D
[anscount] 2,219
[Institute] 의료법인모덕의료재단동민정신건강의학과의원 

****** 4 ******
[Name] 최성환
[Profile] /userinfo/expert/index.nhn?u=c3IUZFm5kpUWmV94ptcMz8UaIIwRRlRZWhqbxzsYPzE%3D
[anscount] 1,533
[Error!] Institute data does not exist.

****** 5 ******
[Name] 김봉수
[Profile] /userinfo/expert/index.nhn?u=tYuqffLqOiT3XXFR7EcoqF5jEJqX8VYYEvjH6cUYo1U%3D
[anscount] 1,406
[Institute] 김봉수의원 원장 

****** 6 ******
[Name] 김슬기
[Profile] /userinfo/expert/index.nhn?u=m6V%2BptYKBpGQrHfBYvzaxAACCmWzKqpDhHsUBMZ3Opw%3

In [ ]:
# 수집된 데이터의 개수가 올바른지 확인
print("- length of doc_names : ", len(doc_names))
print("- length of doc_profiles : ", len(doc_profiles))
print("- length of doc_anscount : ", len(doc_anscount))
print("- length of doc_instits : ", len(doc_instits))

# 리스트를 잘라서 1개 이상의 답변을 가진 의사들의 정보로 필터링
names = doc_names[:84]
profiles = doc_profiles[:84]
anscount = doc_anscount[:84]
instits = doc_instits[:84]

#변경된 데이터 확인
print("-------------------------------------")
print("- length of filtered doc_names : ", len(names))
print("- length of filtered doc_profiles : ", len(profiles))
print("- length of filtered anscount : ", len(anscount))
print("- length of filtered doc_instits : ", len(instits))

# 쉽게 관리하기 위해 데이터 프레임으로 변환
doctor = pd.DataFrame(
                {'name' : names,
                 'profile' : profiles,
                 'anscount' : anscount,
                 'institute' : instits
                })

# 완성된 데이터 확인

- length of doc_names :  90
- length of doc_profiles :  90
- length of doc_anscount :  90
- length of doc_instits :  90
-------------------------------------
- length of filtered doc_names :  84
- length of filtered doc_profiles :  84
- length of filtered anscount :  84
- length of filtered doc_instits :  84


In [ ]:
doctor

,name,profile,anscount,institute
0,신홍범,/userinfo/expert/index.nhn?u=VYRdkbmstCwDtA0tX...,"23,624",코슬립수면의원 대표원장
1,권순모,/userinfo/expert/index.nhn?u=QbPYSQHbEVHsy13fr...,"3,626",
2,배성범,/userinfo/expert/index.nhn?u=HaNfIgmblpoaWZLW%...,"2,219",의료법인모덕의료재단동민정신건강의학과의원
3,최성환,/userinfo/expert/index.nhn?u=c3IUZFm5kpUWmV94p...,"1,533",
4,김봉수,/userinfo/expert/index.nhn?u=tYuqffLqOiT3XXFR7...,"1,406",김봉수의원 원장
5,김슬기,/userinfo/expert/index.nhn?u=m6V%2BptYKBpGQrHf...,"1,264",
6,한경호,/userinfo/expert/index.nhn?u=lXzSACRBbnHB4epsE...,845,탑정신건강의학과의원 원장
7,황선희,/userinfo/expert/index.nhn?u=yaW6qySX89rRGjhXv...,758,해맑은소아정신과 원장
8,최인광,/userinfo/expert/index.nhn?u=qcZLsqvG%2BT3sMqU...,744,최인광정신건강의학과의원 전문의
9,박헌구,/userinfo/expert/index.nhn?u=I1g%2BwFveKTtn1Da...,725,


## 2-2. Crawling question list
각 의사의 프로필 페이지에 들어가 답변 목록들을 크롤링해오자.

In [ ]:
# home link : href로 추출한 링크 앞에 붙여줄 부분
home_link = "https://kin.naver.com"

# 순회할 프로필 링크는 다음과 같이 조회
doctor['profile']

0     /userinfo/expert/index.nhn?u=VYRdkbmstCwDtA0tX...
1     /userinfo/expert/index.nhn?u=QbPYSQHbEVHsy13fr...
2     /userinfo/expert/index.nhn?u=HaNfIgmblpoaWZLW%...
3     /userinfo/expert/index.nhn?u=c3IUZFm5kpUWmV94p...
4     /userinfo/expert/index.nhn?u=tYuqffLqOiT3XXFR7...
                            ...                        
79    /userinfo/expert/index.nhn?u=yJJHuHLWr0wezOovb...
80    /userinfo/expert/index.nhn?u=kReFhSwQrDWBt3uKM...
81    /userinfo/expert/index.nhn?u=bAAJf3SXE7Tu2Xou9...
82    /userinfo/expert/index.nhn?u=0XTZC3pi71irSfQlD...
83    /userinfo/expert/index.nhn?u=b%2Fjex40xRMR%2Bd...
Name: profile, Length: 84, dtype: object

In [ ]:
# 의사별 코드 nhn?u={}
# https://kin.naver.com/userinfo/expert/index.nhn?u=VYRdkbmstCwDtA0tX7VxvE3j7JfcoiA%2Fvj0ZoFMcqks%3D   # 신홍범 의사 프로필
# https://kin.naver.com/userinfo/expert/answerList.nhn?u={doctor_code}
doctor_codes = []
for i in range(len(doctor)):
    doctor_code = doctor['profile'][i].split('/userinfo/expert/index.nhn?u=')[1]
    doctor_codes.append(doctor_code)

print(len(doctor_codes))

84


In [ ]:
# 예진 수정
# 이곳에 질문의 제목과, 그 링크를 담을 예정

li_title = []
li_doctor = []
li_date = []
li_cate = []
li_link = []

for i, doctor_code in enumerate(doctor_codes):
    print("\n---------------------------",i+1,"---------------------------")
    url = f'https://kin.naver.com/userinfo/expert/answerList.nhn?u={doctor_code}'
    print(url)
    time.sleep(5)
    response = requests.get(url) 

    # 링크 크롤링
    # 최대 페이지 1179, max를 1500으로 지정함.
    for pages in range(1, 1500):
        print("\n********", pages, "page ********")
        temp = f'https://kin.naver.com/userinfo/expert/answerList.nhn?u={doctor_code}&page={pages}'
        print("Move to this url : ", temp)
        response = requests.get(temp)
        
        # 한 페이지씩 이동하면서 연결
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            try:
                for j in range(1, 21):
                    title = soup.select_one("#au_board_list > tr:nth-of-type(" + str(j) + ") > td.title > a")
                    category = soup.select_one("#au_board_list > tr:nth-of-type(" + str(j) + ") > td.field > a")
                    date = soup.select_one("#au_board_list > tr:nth-of-type(" + str(j) + ") > td.t_num.tc")
                    link = title.get('href')

                    # Remove white space
                    title = title.get_text().strip()
                    category = category.get_text().strip()
                    date = date.get_text().strip()

                    # Into list
                    li_title.append(title)
                    li_cate.append(category)
                    li_date.append(date)
                    li_link.append(link)
                    li_doctor.append(doctor['name'][i])
                    
                    print("-[Success!]", title, "|", category, "|", date, "|", link)

            # 본래 한 페이지에는 20개의 목록이 존재, 만약에 부족하다면 모든 데이터를 다 긁어왔다고 볼 수 있음. 
            # 따라서 break로 반복문에서 나간 이후, 새로운 의사의 페이지로 이동.
            except:
                print("[Error!] This page does not include all items.")
                break;


        else:
            print(response.status_code)
        

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
-[Success!] 이게 가위눌림이 맞나요? | 정신건강의학과 | 2020.06.22. | /qna/detail.nhn?d1id=7&dirId=70109&docId=360428763
-[Success!] 무서운꿈을 꾸는 이유 | 정신건강의학과 | 2020.06.22. | /qna/detail.nhn?d1id=7&dirId=70109&docId=360424122
-[Success!] 요즘 너무 피곤합니다 | 정신건강의학과 | 2020.06.22. | /qna/detail.nhn?d1id=7&dirId=70109&docId=360337740
-[Success!] 기면증이 있는 것 같아요 | 외과 | 2020.06.22. | /qna/detail.nhn?d1id=7&dirId=70103&docId=360292788
-[Success!] 가위눌림 스트레스 현상 환청 | 정신건강의학과 | 2020.06.22. | /qna/detail.nhn?d1id=7&dirId=70109&docId=360280332
-[Success!] 몽유병이랑 잠꼬대 | 정신건강의학과 | 2020.06.19. | /qna/detail.nhn?d1id=7&dirId=70109&docId=360124240
-[Success!] 선릉 30대후반/남 불면증치료, 수면제 말고... | 정신건강의학과 | 2020.06.18. | /qna/detail.nhn?d1id=7&dirId=70109&docId=360085173
-[Success!] 몽유병인가요 | 정신건강의학과 | 2020.06.18. | /qna/detail.nhn?d1id=7&dirId=70109&docId=360023427
-[Success!] 선릉 30대초반/남 불면증, 몇년전에 치료를... | 정신건강의학과 | 2020.06.17. | /qna/detail.nhn?d1id=7&dirId=70109&docId=359976197
-[Success!] 선릉 20대후반/여 불면증,

In [ ]:
# 수집된 데이터의 개수가 올바른지 확인
print("- length of li_title : ", len(li_title))
print("- length of li_date : ", len(li_date))
print("- length of li_cate : ", len(li_cate))
print("- length of li_link : ", len(li_link))
print("- length of li_doctor : ", len(li_doctor))

- length of li_title :  39774
- length of li_date :  39774
- length of li_cate :  39774
- length of li_link :  39774
- length of li_doctor :  39774


In [ ]:
# 아까 의사 정보 수집할 때 같이 해두었던 answer count 숫자의 총합
cnt_sum = 0
for i in range(len(doctor['anscount'])):
    cnt_sum += int(doctor['anscount'][i].replace(',',''))
cnt_sum

49363

In [ ]:
# 쉽게 관리하기 위해 데이터 프레임으로 변환
question_list = pd.DataFrame(
                {'title' : li_title,
                 'date' : li_date,
                 'category' : li_cate,
                 'url' : li_link
                })

# 완성된 데이터 확인
question_list

,title,date,category,url
0,수면무호흡증 심장통증..심뇌혈관질환 관련...,2020.12.24.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37677...
1,수면시간이 길고 꿈을 계속 꿉니다 ㅠㅠ,2020.12.24.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37669...
2,수면센터 가보려고 하는데요 수면장애치료...,2020.12.23.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37667...
3,몽유병인지?,2020.12.23.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37663...
4,불면증 치료 어떻게하나요,2020.12.22.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37652...
...,...,...,...,...
39769,공황장애 불안장애,2020.10.13.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37059...
39770,명인벤즈트로핀메실레이트정2mg,2020.10.13.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37059...
39771,선천적으로 토를 너무 자주 하는 것,2018.07.17.,내과,/qna/detail.nhn?d1id=7&dirId=70101&docId=30554...
39772,매일 우울하진 않지만 이거 우울증인가요?,2019.12.09.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=34216...


In [ ]:
# 저장 후 불러오기
question_list.to_csv("question_list.csv", index=False)

NameError: ignored

In [10]:
question_list = pd.read_csv("data/question_list.csv", encoding='utf-8')

In [11]:
question_list

,title,date,category,url
0,수면무호흡증 심장통증..심뇌혈관질환 관련...,2020.12.24.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37677...
1,수면시간이 길고 꿈을 계속 꿉니다 ㅠㅠ,2020.12.24.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37669...
2,수면센터 가보려고 하는데요 수면장애치료...,2020.12.23.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37667...
3,몽유병인지?,2020.12.23.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37663...
4,불면증 치료 어떻게하나요,2020.12.22.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37652...
...,...,...,...,...
39769,공황장애 불안장애,2020.10.13.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37059...
39770,명인벤즈트로핀메실레이트정2mg,2020.10.13.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37059...
39771,선천적으로 토를 너무 자주 하는 것,2018.07.17.,내과,/qna/detail.nhn?d1id=7&dirId=70101&docId=30554...
39772,매일 우울하진 않지만 이거 우울증인가요?,2019.12.09.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=34216...


## 2-3. Crawling Question and Answer
url 리스트도 만들었으니, 질문과 답변을 가져오자!

In [13]:
filter_word = ['신홍범','권순모','배성범','최성환','김봉수','김슬기','한경호','황선희','최인광','이승훈',
                '박헌구','박태영','박정수','유은정','장현채','신재현','최원석','이춘수','권용석','성기수',
                '이재원','이상수','김만희','김재옥','신상헌','이형곤','윤충한','박준혁','최영훈','손홍석',
                '김윤석','김상은','정성호','이희창','임대창','임찬영','조연수','박준헌','김세웅','한명훈',
                '정우열','김동현','이우철','성종호','조현식','이상경','송미선','강성민','이정석','서경란',
                '김은지','이종헌','임준석','이경수','서효석','김선아','조우동','백용수','김수룡','박준현',
                '유지희','안경진','안현웅','고은상','박종석','최정식','오윤정','한혜성','김성재','최순호',
                '김강률','김남준','김현철','이현제','박병선','이지혜','이희상','장혜련','최상헌','윤혜연',
                  '한병득','이동한','박지웅','정정엽', '하이닥네이버', '신홍법']
# 저장할 리스트
titles = []
questions = []
answers = []
qa_urls = []
failed_urls = []

# 임시 데이터 셋 이용
for i, u in enumerate(question_list.url):
    if i % 100 == 0:
        print("---------------------------",i,"---------------------------")
    url = 'https://kin.naver.com' + u
    response = requests.get(url)

    # candidate answer lists
    candid_answers = []

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
      
        try:
            # title and question
            title = soup.select_one("#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__title > div > div").get_text().strip()
            question = soup.select_one("#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__content").get_text().strip()

        except:
            try: # 제목이 두줄 이상인 것들 default-old -> multiple-old
                title = soup.select_one("#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--multiple-old > div.c-heading__title > div > div").get_text().strip()
                question = soup.select_one("#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--multiple-old > div.c-heading__content").get_text().strip()
            except:
                try: # 제목에 질문 내용 쓴 경우, 어차피 제목은 안쓰니까 동일한 문구로 채우고,제목 내용을 질문에 저장함.
                    title = "same as question"
                    question = soup.select_one("#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--multiple > div.c-heading__title > div > div").get_text().strip()
                except:
                    try: # 제목에 질문 내용이 있는데 한 줄 짜리라서 포맷이 또 달라진 경우 
                        title = "same as question"
                        question = soup.select_one("#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default > div.c-heading__title > div")
                    except:
                        print("[Error!] cannot crawling title and question in this url\n", url, "\n")
                        failed_urls.append(question_list['url'][i])

        try:
            # candidate answers
            for i in range(1,15):
              try:
                temp_answer = soup.select_one("#answer_" + str(i))
                temp_answer = temp_answer.get_text().strip()
                candid_answers.append(temp_answer)
              except:
                break
            
            # find expert answer
            for i in range(len(candid_answers)):
                if any(w in candid_answers[i] for w in filter_word):
                    try:
                        answer = candid_answers[i].split('\n\n\n\n\n\n')[2]
                    except:
                        answer = candid_answers[i]

                    # save data
                    titles.append(title)
                    questions.append(question)
                    answers.append(answer)
                    qa_urls.append(u)
 
            
            # check the answer
            n_cnt = 0
            if len(candid_answers) >= 5:
                n_cnt += 1
                print("number of candidate answer : ", len(candid_answers))

        except:
            print("[Error!] cannot crawling data in this url\n", url, "\n")
            failed_urls.append(question_list['url'][i])

    else:
        print(response.status_code)

print("5개 이상의 답변을 가진 질문 수 : ", n_cnt)

--------------------------- 0 ---------------------------
--------------------------- 100 ---------------------------
--------------------------- 200 ---------------------------
--------------------------- 300 ---------------------------
--------------------------- 400 ---------------------------
--------------------------- 500 ---------------------------
--------------------------- 600 ---------------------------
--------------------------- 700 ---------------------------
--------------------------- 800 ---------------------------
--------------------------- 900 ---------------------------
--------------------------- 1000 ---------------------------
--------------------------- 1100 ---------------------------
--------------------------- 1200 ---------------------------
--------------------------- 1300 ---------------------------
--------------------------- 1400 ---------------------------
--------------------------- 1500 ---------------------------
--------------------------- 1600 ---

In [14]:
# Check saved data
print(len(titles))
print(len(questions))
print(len(answers))
print(len(qa_urls))
print(len(failed_urls))

33305
33305
33305
33305
0


In [15]:
question_list

,title,date,category,url
0,수면무호흡증 심장통증..심뇌혈관질환 관련...,2020.12.24.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37677...
1,수면시간이 길고 꿈을 계속 꿉니다 ㅠㅠ,2020.12.24.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37669...
2,수면센터 가보려고 하는데요 수면장애치료...,2020.12.23.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37667...
3,몽유병인지?,2020.12.23.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37663...
4,불면증 치료 어떻게하나요,2020.12.22.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37652...
...,...,...,...,...
39769,공황장애 불안장애,2020.10.13.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37059...
39770,명인벤즈트로핀메실레이트정2mg,2020.10.13.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=37059...
39771,선천적으로 토를 너무 자주 하는 것,2018.07.17.,내과,/qna/detail.nhn?d1id=7&dirId=70101&docId=30554...
39772,매일 우울하진 않지만 이거 우울증인가요?,2019.12.09.,정신건강의학과,/qna/detail.nhn?d1id=7&dirId=70109&docId=34216...


In [16]:
ls

'1227data(1)'/  'crawling_1227_colab(2).ipynb'   preprocess.ipynb
 backup/         data/                           preprocess_module2.ipynb
 chromedriver    kin_1227.csv


In [17]:
# Convert List to DataFrame and Save
kin = pd.DataFrame(
                {'title' : titles,
                 'question' : questions,
                 'answer' : answers,  # answers
                 'url' : qa_urls  # 추가
                })
kin.to_csv('kin_1228.csv')

In [18]:
kin

,title,question,answer,url
0,수면무호흡증 심장통증..심뇌혈관질환 관련이 있을 수 있나요?,아버지께서 코골이가 심하십니다.코골이소리 엄청 큰 것은 물론이고요.이번에 본집가서 ...,"안녕하세요, 닥톡-네이버 지식iN 상담의사 신홍범입니다.수면무호흡증의 위험성과 관...",/qna/detail.nhn?d1id=7&dirId=70109&docId=37677...
1,수면시간이 길고 꿈을 계속 꿉니다 ㅠㅠ,꿈을 안꾸는 날이 년에 손꼽을 정도이고 매일 꿈을 꿉니다. 꿈들이 다 기억이 나며 ...,안녕하세요. 닥톡 네이버 지식인 상담의사 신홍범입니...,/qna/detail.nhn?d1id=7&dirId=70109&docId=37669...
2,수면센터 가보려고 하는데요 수면장애치료 받아야 할 것 같아요..,잠을 잘 못자요. 잠들 때 까지도 시간이 오래걸리지만.. 잠들고 나서 새벽에 자꾸 ...,"안녕하세요, 닥톡-네이버 지식iN 상담의사 신홍범입니다. 수면클리닉에 가면 모든 수...",/qna/detail.nhn?d1id=7&dirId=70109&docId=37667...
3,몽유병인지?,"안녕하세요, 곧 24살이 될 사람입니다.다름이 아니라 제가 고등학생 때부터 몽유병인...",안녕하세요. 닥톡 네이버 상담의사 신홍범 입니다.몽유병의 치료 방법...,/qna/detail.nhn?d1id=7&dirId=70109&docId=37663...
4,불면증 치료 어떻게하나요,불면증 약먹어도 괜찮을까요?,안녕하세요. 닥톡 네이버 지식인 상담의사 신홍범입니...,/qna/detail.nhn?d1id=7&dirId=70109&docId=37652...
...,...,...,...,...
33300,공황장애 불안장애,제가 두달 전 공황발작 후 동네 신경정신과에서 처방받은 약을 세네번 먹었구요 근데 ...,안녕하세요? 먼저 공황증상에 대해 설명하기 앞서서 불안장애의 큰 분...,/qna/detail.nhn?d1id=7&dirId=70109&docId=37059...
33301,same as question,"[\n, [\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t...",안녕하세요..질문하신 벤즈트로핀메실레이트정은 이름자체가 성분명입니다...,/qna/detail.nhn?d1id=7&dirId=70109&docId=37059...
33302,same as question,선천적으로 토를 너무 자주 하는 것 같습니다25살 여자 술 담배는 전혀 안합니다학창...,안녕하세요. 하이닥-네이버 지식iN 상담의 이동한 입니다.잦은 구토 증세로 많이 힘...,/qna/detail.nhn?d1id=7&dirId=70101&docId=30554...
33303,매일 우울하진 않지만 이거 우울증인가요?,22살 남자입니다.일단 저는 제가 우울증이라 생각한적이 없습니다. 커오면서 외로움도...,"안녕하세요, 잠실하늘 정신건강의학과 박지웅 원장입니다. 마치 점점 ...",/qna/detail.nhn?d1id=7&dirId=70109&docId=34216...


In [19]:
from IPython.core.display import display, HTML

## 전체 구간을 넓게
display(HTML("<style>.container { width:100% !important; }</style>"))

## 각 컬럼 width 최대로
pd.set_option('display.max_colwidth', -1)

## rows 500
pd.set_option('display.max_rows', 500)

## columns
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [20]:
# load
data = pd.read_csv('kin_1228.csv')
data

,Unnamed: 0,title,question,answer,url
0,0,수면무호흡증 심장통증..심뇌혈관질환 관련이 있을 수 있나요?,"아버지께서 코골이가 심하십니다.코골이소리 엄청 큰 것은 물론이고요.이번에 본집가서 자는데 갑자기 코고는소리가 안나서 깜짝놀라서 가봤더니 막혔던 숨을몰아쉬면서 주무시더라고요.코골이 수면무호흡증 인 것 같습니다.그런데 문제는..요즘들어 아버지께서 평상시에 심장통증 비슷하게 가슴 답답함을자주 느끼신다고 합니다… 쥐어짜는 느낌이라고 하시네요.빨리 말씀하시지 제가 걱정할까봐 이제야 말씀하신 것 같습니다….코골이만 생각했을땐 치료가 필요할까? 생각했는데 코골이수면무호흡증이 심혈관, 뇌혈관질환으로 이어질 가능성이 높다라는 기사를 우연히 보고 걱정이 됩니다.더 정확한 답변을 듣고 싶어서 질문드리는데요.수면무호흡증이 심뇌혈관질환과 관련이 있나요? 혈압도 좀 높으신 편입니다.수면무호흡증치료 어떻게 하는 것이 좋을까요..","안녕하세요, 닥톡-네이버 지식iN 상담의사 신홍범입니다.수면무호흡증의 위험성과 관련해서 여러 가지 이슈가 있습니다.우선은 충분한 잠을 못 자니까 낮 동안에 졸리고 그래서 사고 위험이 높아진다가 있고요.기억력이 떨어진다.인지 기능이 떨어진다.치매가 온다.이런 것도 있습니다.수면무호흡증과 관련된 가장 직접적이고 눈에 보이는 위협이라고 할 수 있는 것이혈압이 높아지면서 심장에 무리가 가고 그래서 부정맥, 심근경색, 심장마비, 돌연사 이런 위험이 있는 거고요.또 뇌혈관에 압력을 높여 가지고 뇌경색이라든지 뇌출혈이라든지 그래서 뇌혈관질환으로 인한 사망 위험을 높인다.라고 되어 있습니다.실제로 수면무호흡증 자체가 잠을 잘 때 숨이 멈추는 거고, 우리 몸에 산소가 부족해지는 거고 그래서 산소를 많이 소모하는 심장과 뇌가 우선적으로 타격을 받는 거고요.그 과정에서 뇌세포, 심장세포가 죽어나가는 거고 그만큼 기능이 떨어지는 거고 그래서 사고가 생기는 겁니다.혈압이 높아지는 거 그로 인한 혈관의 내피 부분에 손상이 생기고 혈관 암벽이 다치니까 그 부분이 치유되는 과정에서 두꺼워지죠.막히게 되고 그리고 수면무호흡이 있을 때 심장이 상당히 불규칙하게 됩니다.빨리 뛰다, 느리게 뛰다, 빨리 뛰다, 느리게 뛰다 되는 게 반복되는 거죠.우리가 잠을 잘 때 사실 심장이 쉬어야 되거든요.좀 느리게 안정적으로 뛰면서 쉬는데 그럼 휴식을 못 취하는 거죠.또 낮에는 낮대로 또 활동을 해야 되고 이 과정에서 심장이 빨리 상하는 겁니다.또 뇌혈관질환이 생긴 이유 중에 하나가 경동맥이 있습니다.우리 심장에서 목을 타고 뇌로 피가 공급이 되는데, 목 속에 있는 동맥이죠.그 동맥도 안쪽 부분이 손상을 당합니다.그래서 그 부분에 아무는 과정에서 피떡이 생기는 거죠.그게 붙어있다가 어느 순간 떨어져 나가면서 뇌혈관으로 퍼져 나가고, 가는 뇌혈관을 막아버리는 겁니다.그래서 뇌경색이 생길 수 있죠.이런 형태로 코골이 수면무호흡증이 심장질환, 뇌혈관질환을 유발하게 됩니다.또 당연히 코골이 수면무호흡증을 양압기로 치료해서 특히 수면무호흡으로 인한 피해를 줄여줬을 때 그때 혈압이 떨어지고 또 심장 부정맥도 줄어들고 그래서 심장질환, 뇌혈관질환으로 사망하게 되는 위험이 많이 줄어들더라.라는 연구가 여럿 나와 있습니다.그래서 우리가 인과관계도 확실하고 양압기로 치료를 했을 때 기대수명도 늘어난다.이런 것들을 알게 되었습니다.그래서 심혈관질환, 뇌혈관질환을 예방하기 위해서도 코골이 수면무호흡증 양압기 치료가 필요하고요.이미 심장질환 뇌혈관질환을 겪었거나,겪고 계신 분들의 경우에도 더 악화되는 걸 막기 위해서 수면무호흡증에 대한 정확한 진단과 치료가 필요합니다.이상입니다.\n\n#수면무호흡심뇌혈관질환\n#수면무호흡증\n#수면무호흡증치료\n#수면무호흡증심혈관질환\n#수면무호흡증뇌혈관질환\n#동영상답변\n#지식iN동영상답변\n\n\n출처http://www.doctalk.co.kr/counsel/view/c-sk4pVHD-g9-xjsy",/qna/detail.nhn?d1id=7&dirId=70109&docId=376772588
1,1,수면시간이 길고 꿈을 계속 꿉니다 ㅠㅠ,꿈을 안꾸는 날이 년에 손꼽을 정도이고 매일 꿈을 꿉니다. 꿈들이 다 기억이 나며 꿈을 꾸는 날은 항상 10시간 넘게 자고 만약 그 이하로 자면 몸이 너무 피곤합니다.잠을 줄이고 싶은데 잠을 줄이기가 쉽지않습니다. 꿈을 꾸지않는 날은 잠을 적게자는 것 같은데.. 저한테 수면장애가 있는 것 일까요 ㅠㅠ? 저도 적게자고 개운한 삶을 살고싶습니다.. 참고로 20대입니다,안녕하세요. 닥톡 네이버 지식인 상담의사 신홍범입니다. 저는 매일 밤 꿈을 꿉니다. 참 괴롭습니다. 이렇게 얘기 하시는 분이 있습니다. 이런 분들은 우선 잠은 좀 주무시는 거죠. 잠드는 거 좀 가능한데 잠이 들었을 때 잠이 깊지 않고 자다가 수시로 깨거나 꿈을 경험하게 되는 거죠. 기본적으로 어떤 분이 저는 꿈을 많이 꾸어요. 라고 얘기하면 수면의학자인 저는 이걸 “저의 수면의 질이 많이 좋지 않습니다” 라고 번역해서 듣습니다. 그런 뜻입니다. 사람은 누구나 꿈을 꾸어요. 전체 수면에 4분의 1이 꿈이거든요. 그걸 깊고 양질의 잠을 자는 사람들은 기억하지 못합니다. 그게 정상이에요. 근데 잠을 얕게 사람들은 그걸 기억하게 되기가 쉽습니다. 그래서 나는 꿈을 많이 꾼다라고 스스로 얘기하시는 거죠. 그래서 중요한 거는 꿈 자체를 없애는 게 아니라 꿈을 많이 꿀 수밖에 없는 꿈을 많이 경험할 수밖에 없는 수면을 방해하는 수면 질환을 찾아서 치료 하는 겁니다. 어떤 수면 질환이 있는지 우리가 알기 위해서 수면다원검사라는게 있습니다. 하룻밤 주무시면서 잠을 찍어 보는 겁니다. 마치 우리가 뼈가 부러졌다. 그러면 엑스레이를 찍어서 진단을 하는 거와 마찬가지로 수면의 질을 수면에 문제를 수면다원검사를 통해서 진단할 수 있고 거기에 따라서 치료 계획을 세우게 됩니다. \n\n#동영상답변\n#지식iN동영상답변,/qna/detail.nhn?d1id=7&dirId=70109&docId=376699336
2,2,수면센터 가보려고 하는데요 수면장애치료 받아야 할 것 같아요..,잠을 잘 못자요. 잠들 때 까지도 시간이 오래걸리지만.. 잠들고 나서 새벽에 자꾸 깨서 다시 잠 못자는걸 거의 반년을 겪은 것 같네요… 잠을 못자니까 살도 빠지고 아픈사람처럼 변한 것 같아요... 잠에 문제가 있다고 생각이 들어요. 그래서 수면센터 가보고 싶어요.. 수면센터 가면 수면장애 원인 같은걸 다 찾을 수가 있는건가요? 원래 자는것에 문제가 없었는데 갑자기 이렇게 변한거라 수면장애원인 뭔지 궁금해요… 수면만 .. 보는 병원이 수면센터 라는데 맞지요? 수면센터 가보면 수면장애치료 잘 받아볼 수 있겠죠? 문제없이 잘 자고싶어요…,"안녕하세요, 닥톡-네이버 지식iN 상담의사 신홍범입니다. 수면클리닉에 가면 모든 수면 질환을 다 진단할 수 있느냐? 치료할 수 있느냐?라고 궁금해하시는 분들이 있습니다. 수면클리닉은 문자 그대로 수면질환에 대한 종합적인 진료가 이루어지는 곳입니다. 단순히 이비인후과, 정신과, 신경과, 내과, 소아과가 아니라요. 수면에 대한 종합적인 진료가 가능한 곳이죠. 진료에는 물론 진찰을 하는 것도 있고, 검사를 하는 것도 있고, 거기에 맞는 치료도 있어야지 됩니다. 또 수면클리닉에서 진단하는 질환으로는 불면증, 코골이 수면무호흡증, 기면증 하지불안증, 잠꼬대, 몽유병, 렘수면장애, 교대 근무로 인한 수면문제 등. 대부분의 수면질환을 다 진단하고 치료할 수 있습니다. 예를 들어 이갈이도 진단할 수 있죠. 그리고 그에 맞는 치료도 같이 제공합니다. 수면클리닉에서 수면질환 진단을 위해서는 수면다원검사를 주로 하게 되고요. 기면증 진단을 위해서는 주간 검사도 같이 또 할 수 있습니다. 또, 하지불안증이 있을 때는 운동 억제 검사를 하는 경우도 있습니다. 이런 검사를 통해서 질환에 심한 정도를 파악하고 약물이나 또 다른 형태 양압기를 이용한 수면무호흡증 치료 등과 같이 또, 경우에 따라서는 불면증의 비약물 치료로 인지행동치료 등등을 병행을 해서 그 사람의 수면의 질을 개선하고 수면질환을 치료한다. 그런 곳이 수면클리닉이다. 이렇게 이해하시면 되겠습니다. 이상입니다. 